## 더미데이터 생성

In [46]:
import csv
import random
from datetime import datetime, timedelta

def generate_transaction_data(num_rows=1000):
    merchants = [
        '롯데월드', '세븐일레븐', '이마트', '맥도날드', '세인트존스호텔',
        'GS25', '강동성심병원', '롯데리아', '카카오프렌즈', '롯데시네마',
        'CGV', '코스트코', '스타벅스', '런던베이글', '강동도서관',
        '오렌지주유소', '더벤티', '쿰베오', '메가커피', '은혜공인중개사무소', '정약국'
    ]

    amount_ranges = {
        '롯데월드': (50000, 150000),
        '세븐일레븐': (1000, 15000),
        '이마트': (5000, 45000),
        '맥도날드': (3000, 20000),
        '세인트존스호텔': (80000, 150000),
        'GS25': (1000, 12000),
        '강동성심병원': (30000, 200000),
        '롯데리아': (2000, 15000),
        '카카오프렌즈': (5000, 25000),
        '롯데시네마': (8000, 25000),
        'CGV': (8000, 25000),
        '코스트코': (20000, 80000),
        '스타벅스': (4000, 15000),
        '런던베이글': (3000, 25000),
        '강동도서관': (1000, 5000),
        '오렌지주유소': (40000, 120000),
        '더벤티': (3000, 12000),
        '쿰베오': (8000, 20000),
        '메가커피': (2000, 12000),
        '은혜공인중개사무소': (500000, 2000000),
        '정약국': (5000, 30000)
    }
    start_date = datetime(2025, 6, 1)
    end_date = datetime(2025, 8, 31)
    date_range = (end_date - start_date).days

    # CSV 파일 생성
    with open('member.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['memberId', 'amount', 'merchant', 'date'])

        for i in range(num_rows):
            # memberId는 1-10 순환
            member_id = (i % 10) + 1

            merchant = random.choice(merchants)

            min_amount, max_amount = amount_ranges[merchant]
            amount = random.randint(min_amount, max_amount)

            variation = random.uniform(0.8, 1.2)
            amount = int(amount * variation)

            random_days = random.randint(0, date_range)
            transaction_date = start_date + timedelta(days=random_days)
            date_str = transaction_date.strftime('%Y-%m-%d')

            writer.writerow([member_id, amount, merchant, date_str])

    print(f"member.csv 파일 생성 완료 ({num_rows}행)")

if __name__ == '__main__':
    generate_transaction_data(1000)

member.csv 파일 생성 완료 (1000행)


## 데이터 구성 확인

In [47]:
import pandas as pd

# CSV 파일 경로 지정
file_path = 'member.csv'

# CSV 불러오기
df = pd.read_csv(file_path)

# 상위 5개 행 출력
print(df.head())

   memberId  amount merchant        date
0         1   21981      CGV  2025-08-17
1         2    1531    세븐일레븐  2025-06-23
2         3   14947    세븐일레븐  2025-08-06
3         4   94926   강동성심병원  2025-08-31
4         5   13423     스타벅스  2025-08-04


## 상점명 기반 카테고리 추출

In [48]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os

api_key = os.getenv("KAKAO_API_KEY")
headers = {"Authorization": f"KakaoAK {api_key}"}

def get_category_group(merchant_name):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    response = requests.get(url, headers=headers, params={"query": merchant_name})
    if response.status_code == 200:
        docs = response.json().get("documents")
        if docs and docs[0].get("category_group_name"):
            category = docs[0]["category_group_name"]
            if not category:
                return "기타"
            return category
    return "기타"

# CSV파일 읽기
df = pd.read_csv("member.csv")

# 대분류 카테고리(공식 카테고리 그룹명) 추가
df['category'] = df['merchant'].apply(get_category_group)

# CSV 저장
df.to_csv("member_with_category.csv", index=False)
print(df.head())


   memberId  amount merchant        date category
0         1   21981      CGV  2025-08-17     문화시설
1         2    1531    세븐일레븐  2025-06-23      편의점
2         3   14947    세븐일레븐  2025-08-06      편의점
3         4   94926   강동성심병원  2025-08-31       병원
4         5   13423     스타벅스  2025-08-04       카페


## 카테고리별 MCC 수동 매핑

In [49]:
import pandas as pd

# 카카오 기준에 맞춰 MCC 수동 매핑
mcc_mapping = {
    "대형마트": "5300",
    "편의점": "5499",
    "주차장": "7523",
    "주유소, 충전소": "5541",
    "지하철역": "4111",
    "문화시설": "7991",
    "중개업소": "6536",
    "공공기관": "9399",
    "관광명소": "7991",
    "숙박": "7011",
    "음식점": "5812",
    "카페": "5814",
    "병원": "8062",
    "약국": "5912"
}

# CSV 파일 불러오기
df = pd.read_csv('member_with_category.csv')

# category 컬럼(대분류명)을 기반으로 MCC 코드를 매핑해 새 컬럼 생성
df['mcc'] = df['category'].map(mcc_mapping).fillna('0000')  # 미매핑시 '0000' 처리

# 결과 저장
df.to_csv('member_with_category_mcc.csv', index=False)

print(df.head())



   memberId  amount merchant        date category   mcc
0         1   21981      CGV  2025-08-17     문화시설  7991
1         2    1531    세븐일레븐  2025-06-23      편의점  5499
2         3   14947    세븐일레븐  2025-08-06      편의점  5499
3         4   94926   강동성심병원  2025-08-31       병원  8062
4         5   13423     스타벅스  2025-08-04       카페  5814


## MCC 기반 탄소 배츌량 매핑

In [50]:
import pandas as pd
import requests
import uuid

# 1) member_with_category_mcc 파일을 같은 폴더에 두고 불러오기
df = pd.read_csv('member_with_category_mcc.csv')
from dotenv import load_dotenv
load_dotenv()
import os

# 2) demo API URL
API_URL = os.getenv("CARBON_API_URL")

# 3) 각 행마다 demo 호출 및 carbonEmissionInOunces 수집
carbon_list = []
for _, row in df.iterrows():
    tx_id = str(uuid.uuid4())
    payload = [
        {
            "transactionId": tx_id,
            "amount": {
                "value": str(row['amount']),
                "currencyCode": "KRW"
            },
            "transactionDate": row['date'],
            "mcc": int(row['mcc']),
            "merchantCountryCode": "KOR",
            "paymentNetwork": "MASTERCARD",
            "cardBrand": "OTH"
        }
    ]
    resp = requests.post(API_URL, json=payload)
    if resp.status_code == 200:
        data = resp.json()[0]
        carbon_value = data.get('carbonEmissionInOunces')
        # None이나 Null인 경우 0으로 대체
        if carbon_value is None:
            carbon_value = 0
        carbon_list.append(carbon_value)
    else:
        carbon_list.append(0)  # 에러 시에도 0으로 처리

# 4) 원본 DataFrame에 컬럼 추가 (컬럼명 'carbon'으로 변경)
df['carbon'] = carbon_list

# 5) final_member.csv 로 저장
df.to_csv('final_member.csv', index=False)

print(df.head())

   memberId  amount merchant        date category   mcc  carbon
0         1   21981      CGV  2025-08-17     문화시설  7991   65.01
1         2    1531    세븐일레븐  2025-06-23      편의점  5499    8.80
2         3   14947    세븐일레븐  2025-08-06      편의점  5499   85.94
3         4   94926   강동성심병원  2025-08-31       병원  8062  387.43
4         5   13423     스타벅스  2025-08-04       카페  5814  147.18


## 데이터 기반 탄소 배출 분석

In [55]:
import pandas as pd
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

columns = ['memberId', 'amount','merchant','date','category','mcc','carbon']
df = pd.read_csv('final_member.csv', names=columns, header=None)

df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce')

df['amount'] = pd.to_numeric(df['amount'], errors='coerce').fillna(0)

# --- 함수 1: 기간별 멤버 탄소 합산 및 등수 ---
def get_rank_by_period(df, start_date, end_date):
    mask = (df['date'] >= pd.to_datetime(start_date)) & (df['date'] <= pd.to_datetime(end_date))
    df_period = df.loc[mask]

    # 멤버별 탄소 합산
    carbon_sum = df_period.groupby('memberId')['amount'].sum()

    # 탄소 합산 기준 오름차순 등수 (탄소 적을수록 1등)
    ranks = carbon_sum.rank(method='dense', ascending=True).astype(int)

    # 결과 DataFrame 생성: carbon_sum 과 ranks 를 합치고 memberId를 컬럼으로 변환
    result = pd.DataFrame({
        'memberId': carbon_sum.index,
        'carbon_sum': carbon_sum.values,
        'rank': ranks.values
    }).sort_values('rank').reset_index(drop=True)

    return result


# --- 함수 2: 멤버별 월/주/일 단위 고탄소 및 저탄소 소비 업종 Top3 추출 ---
def get_top3_categories_by_period(df, start_date, end_date):
    mask = (df['date'] >= pd.to_datetime(start_date)) & (df['date'] <= pd.to_datetime(end_date))
    df_period = df.loc[mask].copy()

    # 기타 카테고리 제외
    df_period = df_period[df_period['category'] != '기타']

    # carbon 컬럼을 숫자형으로 강제 변환 (안전 조치)
    df_period['carbon'] = pd.to_numeric(df_period['carbon'], errors='coerce').fillna(0)

    # 멤버별 카테고리별 합계 계산
    grouped = df_period.groupby(['memberId', 'category'])['carbon'].sum().reset_index()

    # 고탄소 상위 3개, 저탄소 하위 3개 추출
    top3_high = grouped.groupby('memberId', group_keys=False) \
                   .apply(lambda x: x.nlargest(3, 'carbon')) \
                   .reset_index(drop=True)
    top3_low = grouped.groupby('memberId', group_keys=False) \
                  .apply(lambda x: x.nsmallest(3, 'carbon')) \
                  .reset_index(drop=True)

    return top3_high, top3_low


if __name__ == '__main__':
    # 원하는 기간 입력
    start_date = '2025-06-05'
    end_date = '2025-07-31'

    period_rank = get_rank_by_period(df, start_date, end_date)
    print(f"=== 기간별 멤버 탄소 합계 및 등수 ({start_date} ~ {end_date}) ===")
    print(period_rank.head(5).to_string(index=False))

    top3_high, top3_low = get_top3_categories_by_period(df, start_date, end_date)

    print(f"\n=== 기간({start_date} ~ {end_date}) 내 멤버별 고탄소 소비 업종 Top 3 ===")
    print(top3_high.head(5).to_string(index=False))

    print(f"\n=== 기간({start_date} ~ {end_date}) 내 멤버별 저탄소 소비 업종 Top 3 ===")
    print(top3_low.head(5).to_string(index=False))

    # 현재 기준 일/주/월별 통계
    today = pd.to_datetime('today').normalize()

    # 일별 (오늘)
    day_start = today
    day_end = today
    rank_day = get_rank_by_period(df, day_start, day_end)
    high_day, low_day = get_top3_categories_by_period(df, day_start, day_end)
    print(f"\n=== 일별 탄소 합계 및 등수 ({day_start.date()}) ===")
    print(rank_day.head(5).to_string(index=False))
    print(f"\n=== 일별 고탄소 소비 업종 Top 3 ({day_start.date()}) ===")
    print(high_day.head(5).to_string(index=False))
    print(f"\n=== 일별 저탄소 소비 업종 Top 3 ({day_start.date()}) ===")
    print(low_day.head(5).to_string(index=False))

    # 주별 (최근 7일)
    week_start = today - pd.Timedelta(days=6)
    week_end = today
    rank_week = get_rank_by_period(df, week_start, week_end)
    high_week, low_week = get_top3_categories_by_period(df, week_start, week_end)
    print(f"\n=== 주별 탄소 합계 및 등수 ({week_start.date()} ~ {week_end.date()}) ===")
    print(rank_week.head(5).to_string(index=False))
    print(f"\n=== 주별 고탄소 소비 업종 Top 3 ({week_start.date()} ~ {week_end.date()}) ===")
    print(high_week.head(5).to_string(index=False))
    print(f"\n=== 주별 저탄소 소비 업종 Top 3 ({week_start.date()} ~ {week_end.date()}) ===")
    print(low_week.head(5).to_string(index=False))

    # 월별 (최근 30일)
    month_start = today - pd.Timedelta(days=29)
    month_end = today
    rank_month = get_rank_by_period(df, month_start, month_end)
    high_month, low_month = get_top3_categories_by_period(df, month_start, month_end)
    print(f"\n=== 월별 탄소 합계 및 등수 ({month_start.date()} ~ {month_end.date()}) ===")
    print(rank_month.head(5).to_string(index=False))
    print(f"\n=== 월별 고탄소 소비 업종 Top 3 ({month_start.date()} ~ {month_end.date()}) ===")
    print(high_month.head(5).to_string(index=False))
    print(f"\n=== 월별 저탄소 소비 업종 Top 3 ({month_start.date()} ~ {month_end.date()}) ===")
    print(low_month.head(5).to_string(index=False))

=== 기간별 멤버 탄소 합계 및 등수 (2025-06-05 ~ 2025-07-31) ===
memberId  carbon_sum  rank
       8   1659179.0     1
       9   3236342.0     2
       7   3368314.0     3
       6   3657545.0     4
       2   4457962.0     5

=== 기간(2025-06-05 ~ 2025-07-31) 내 멤버별 고탄소 소비 업종 Top 3 ===
memberId category  carbon
       1     대형마트 1673.20
       1      음식점 1450.98
       1       카페 1195.45
      10       숙박 4076.28
      10       병원 1609.28

=== 기간(2025-06-05 ~ 2025-07-31) 내 멤버별 저탄소 소비 업종 Top 3 ===
memberId category  carbon
       1  주유소,충전소    0.00
       1     중개업소    0.00
       1       약국   59.86
      10  주유소,충전소    0.00
      10     중개업소    0.00

=== 일별 탄소 합계 및 등수 (2025-08-07) ===
memberId  carbon_sum  rank
       9      6936.0     1
       8     17161.0     2
       7     19248.0     3
       5     26013.0     4
       1     30301.0     5

=== 일별 고탄소 소비 업종 Top 3 (2025-08-07) ===
memberId category  carbon
       1      음식점  240.50
       1      편의점   48.11
       2       약국  167.36
       2     

## 환경 Q&A 챗봇

In [44]:
import os
import json
import requests
from dotenv import load_dotenv

def load_api_key():
    load_dotenv()
    key = os.getenv("GEMINI_API_KEY")
    if not key:
        raise ValueError("GEMINI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
    return key

def generate_environmental_response(user_prompt: str) -> str:
    """
    Gemini API를 호출하여 사용자 프롬프트를
    환경적 지속 가능성 관점에서 분석한 답변을 반환합니다.
    """
    api_key = load_api_key()
    model_id = "gemini-2.5-flash"
    endpoint = f"https://generativelanguage.googleapis.com/v1beta/models/{model_id}:generateContent?key={api_key}"

    system_prompt = """당신은 환경 특화 챗봇입니다. 사용자의 질문을 환경적 지속 가능성 관점에서 분석하고,
    실용적이고 구체적인 조언을 300자 이내로 제공해주세요.  """

    contents = [
        {
            "parts": [
                {"text": system_prompt},
                {"text": f"질문: {user_prompt}"}
            ]
        }
    ]

    payload = {
        "contents": contents,
        "generationConfig": {
            "temperature": 0.7,
            "candidateCount": 1,
            "maxOutputTokens": 2048,
            "topP": 0.8,
            "topK": 40
        },
        "safetySettings": [
            {
                "category": "HARM_CATEGORY_HARASSMENT",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            },
            {
                "category": "HARM_CATEGORY_HATE_SPEECH",
                "threshold": "BLOCK_MEDIUM_AND_ABOVE"
            }
        ]
    }

    headers = {"Content-Type": "application/json; charset=UTF-8"}

    try:
        response = requests.post(endpoint, headers=headers, json=payload, timeout=30)

        if response.status_code != 200:
            raise RuntimeError(f"API 실패: {response.status_code} - {response.text}")

        data = response.json()

        if "candidates" not in data or not data["candidates"]:
            raise ValueError("API 응답에 candidates가 없습니다.")

        candidate = data["candidates"][0]
        finish_reason = candidate.get("finishReason", "UNKNOWN")

        # 텍스트 추출
        result_text = ""

        # 1. 표준 구조: content.parts[].text
        if "content" in candidate and "parts" in candidate["content"]:
            parts = candidate["content"]["parts"]
            if isinstance(parts, list) and len(parts) > 0:
                for part in parts:
                    if isinstance(part, dict) and "text" in part:
                        result_text += part["text"]

        # 2. 직접 text 필드
        if not result_text and "text" in candidate:
            result_text = candidate["text"]

        # 3. content 직속 text 필드
        if not result_text and "content" in candidate and "text" in candidate["content"]:
            result_text = candidate["content"]["text"]

        # 4. 기타 가능한 필드들 확인
        if not result_text:
            possible_text_fields = ["output", "response", "answer", "message", "generated_text"]
            for field in possible_text_fields:
                if field in candidate:
                    result_text = str(candidate[field])
                    break

        return result_text.strip()

    except requests.exceptions.Timeout:
        return "API 요청 시간이 초과되었습니다. 다시 시도해주세요."
    except requests.exceptions.RequestException as e:
        return f"네트워크 오류가 발생했습니다: {str(e)}"
    except json.JSONDecodeError:
        return "API 응답을 파싱할 수 없습니다."
    except Exception as e:
        return f"처리 중 오류가 발생했습니다: {str(e)}"


def test_api_response_structure():
    try:
        test_prompt = "간단한 테스트 질문입니다."
        result = generate_environmental_response(test_prompt)
        return result
    except Exception as e:
        print(f"테스트 실패: {e}")
        return None


if __name__ == '__main__':
    test_result = test_api_response_structure()

    if test_result:
        # 사용자 프롬프트 입력
        user_prompt = "분리수거 하는 법 알려줘"
        answer = generate_environmental_response(user_prompt)
        print(answer)

분리수거는 자원 순환의 핵심입니다. 환경적 지속 가능성을 위해 다음 원칙을 지켜주세요:

1.  **비우고 헹구기:** 내용물을 깨끗이 비우고 물로 헹궈 오염을 제거합니다. (음식물 오염 시 재활용 불가)
2.  **재질별 분리:** 플라스틱, 종이, 유리, 캔 등 재질별로 정확히 나눕니다.
3.  **라벨/뚜껑 분리:** 비닐 라벨, 금속 뚜껑 등 다른 재질은 제거 후 따로 배출합니다.
4.  **압축:** 부피를 줄여 효율적인 운반 및 재활용을 돕습니다.

이 작은 노력이 지구의 부담을 크게 줄입니다.
